In [1]:
from sklearn.metrics import mean_squared_error
from math import sqrt
import csv
import numpy as np
import collections
import copy

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
# Test dataset without rating
test_dataset_without_rating_dir = '/content/drive/MyDrive/Assignment5/test_dataset_without_rating.csv'
# Test dataset
test_dataset_dir = '/content/drive/MyDrive/Assignment5/test_dataset.csv'
# Training dataset
training_dataset_dir = '/content/drive/MyDrive/Assignment5/training_dataset.csv'

# Dictionary to save Test dataset without rating
testDatasetWithoutRating = {}
# Dictionary to save Test dataset
testDataset = {}
# Dictionary to save Training dataset
trainingDataset = {}

###########################################################################################################
# Test dataset without rating
# Data to predict

# create dictionary for test data set prediction
user_x_movie_dict = {}
# extract movie list 
user_x_movie_list = []
# Open file
with open(test_dataset_without_rating_dir) as tdwr:
    # read as csv
    csv_reader = csv.reader(tdwr, delimiter=',')
    line_count = 0
    for row in csv_reader:
        # header
        if line_count == 0:
            print(f'test_dataset_without_rating.csv: header ({row[0]}) ({row[1]}) ({row[2]})')
            line_count += 1
        # data
        else:
            # if key does not exist
            if not int(row[0]) in testDatasetWithoutRating:
                testDatasetWithoutRating[int(row[0])] = dict()
            if not int(row[0]) in user_x_movie_dict:
                user_x_movie_dict[int(row[0])] = []
            
            user_x_movie_dict[int(row[0])].append(int(row[1]))
            # extract user list 
            user_x_movie_list.append(int(row[0]))
            # user | moive = rating initialized as 0
            testDatasetWithoutRating[int(row[0])][int(row[1])] = 0
            line_count += 1
    print(f'Processed {line_count} lines.\n')
# number of users in test dataset    
unique_user_x_list = list(set(user_x_movie_list))

testDatasetWithoutRating_global_base = copy.deepcopy(testDatasetWithoutRating)
testDatasetWithoutRating_adj = copy.deepcopy(testDatasetWithoutRating)
###########################################################################################################
# For rmse accuracy test
# Open file
with open(test_dataset_dir) as tdd:
    # read as csv
    csv_reader = csv.reader(tdd, delimiter=',')
    line_count = 0
    for row in csv_reader:
        # header
        if line_count == 0:
            print(f'test_dataset.csv: header ({row[0]}) ({row[1]}) ({row[2]}) ({row[3]})')
            line_count += 1
        # data
        else:
            # if key does not exist
            if not int(row[0]) in testDataset:
                testDataset[int(row[0])] = dict()
            # user | moive = rating
            testDataset[int(row[0])][int(row[1])] = int(row[2])
            line_count += 1
    print(f'Processed {line_count} lines.\n')
###########################################################################################################
# For training model
# Dictionary of training dataset
training_user_x_movie_list = {}
# Dictionary of training dataset for user_x_global_baseline
training_user_x_global_baseline = {}
# Dictionary of training dataset for item_i_global_baseline
training_item_i_global_baseline = {}
# Open file
with open(training_dataset_dir) as trdd:
    # read as csv
    csv_reader = csv.reader(trdd, delimiter=',')
    line_count = 0
    for row in csv_reader:
        # header
        if line_count == 0:
            print(f'training_dataset.csv: header ({row[0]}) ({row[1]}) ({row[2]}) ({row[3]})')
            line_count += 1
        # data
        else:
            # if key does not exist
            if not int(row[1]) in trainingDataset:
                trainingDataset[int(row[1])] = []
            if not int(row[0]) in training_user_x_movie_list:
                training_user_x_movie_list[int(row[0])] = []
            if not int(row[0]) in training_user_x_global_baseline:
                training_user_x_global_baseline[int(row[0])] = dict()
            if not int(row[1]) in training_item_i_global_baseline:
                training_item_i_global_baseline[int(row[1])] = dict()
            # training_user_x_movie_list: user | list(movie)
            training_user_x_movie_list[int(row[0])].append(int(row[1]))
            # trainingDataset: movie | (user, rating)
            trainingDataset[int(row[1])].append((int(row[0]), int(row[2])))
            # training_user_x_global_baseline: user | movie | rating
            training_user_x_global_baseline[int(row[0])][int(row[1])] = int(row[2])
            # training_user_x_global_baseline: movie | user | rating
            training_item_i_global_baseline[int(row[1])][int(row[0])] = int(row[2])
            line_count += 1
    print(f'Processed {line_count} lines.\n')

test_dataset_without_rating.csv: header (user_id) (item_id) (movie_name)
Processed 5001 lines.

test_dataset.csv: header (user_id) (item_id) (rating) (movie_name)
Processed 5001 lines.

training_dataset.csv: header (user_id) (item_id) (rating) (movie_name)
Processed 95001 lines.



In [4]:
# Make an utility matrix
# row movies column users
# 943 users
# 1682 items
# 100000 ratings

# create utility Matrix initially have rating value as 0
# items 1682 | users 943
utilityMatrix = dict.fromkeys(range(1, 1683), dict())
for k, v in utilityMatrix.items():
    utilityMatrix[k] = dict.fromkeys(range(1, 944), 0)
        
# Fill in the utility matrix with training dataset ratings
for k, v in trainingDataset.items():
    for (user, rating) in v:
        utilityMatrix[k][user] = rating     

In [5]:
# cosine similarty 
def cosineSim(sum_list1, list1, list2):
    # list1 * list2
    sumList = np.dot(list1, list2)
    # if 0, no need for further caculation. Return 0
    if sumList == 0:
        return 0
    # if 0, no need for further caculation. Return 0
    if sum_list1 == 0:
        return 0
    # take square root of sum
    sqr_list1 = sqrt(sum_list1)
    # sum of squared list
    sumlist2 = np.sum(np.square(list2))
    # if 0, no need for further caculation. Return 0
    if sumlist2 == 0:
        return 0
    # take square root of sum
    sqr_list2 = sqrt(sumlist2)
    # similiarity value
    sim = sumList / (sqr_list1 * sqr_list2)

    return sim
# RMSE function
def RMSE(y_actual, y_predicted):
    rms = mean_squared_error(y_actual, y_predicted, squared=False)
    return round(rms,4)

In [6]:
# Create dictionary of similarity matric
simMetric = dict.fromkeys(sorted(unique_user_x_list), dict()) # user 100

for k1, v1 in user_x_movie_dict.items(): # user ID / movie ID
    simMetric[k1] = dict.fromkeys(sorted(v1), dict()) # movieId of test dataset
    for v in v1:
        temp_dict = dict.fromkeys(sorted(training_user_x_movie_list[k1]), 0) #  # movieId of training dataset rated by same user
        if not v in simMetric:
            simMetric[k1][v] = temp_dict
        else:
            simMetric[k1][v].update(temp_dict)

In [7]:
# page table to avoid calculating same similarity again
pageTable = dict()

for k1, v1 in simMetric.items(): # user
    for k2, v2 in v1.items(): # first movie ID, testdata set
        l1 = list(utilityMatrix[k2].values())
        # calculate sum of squared list in advance to acheive faster execution time
        sumlist1 = np.sum(np.square(l1))
        for k3, v3 in v2.items(): # second movie ID, training set 
            # if movie1 and movie2 are same, similiarity will be 1. No need to calculate
            if k2 == k3:
                simMetric[k1][k2][k3] = 1.0
                pageTable[(k2, k3)] = 1.0
                pageTable[(k3, k2)] = 1.0
            # page hit, no need to calculate
            elif (k2, k3) in pageTable and pageTable[(k2, k3)] != 0.0:
                simMetric[k1][k2][k3] = pageTable[(k2, k3)]
            # page hit with inverted indexes, no need to calculate
            elif (k3, k2) in pageTable and pageTable[(k3, k2)] != 0.0:
                simMetric[k1][k2][k3] = pageTable[(k3, k2)]
            # no hit. Calculation needed
            else:
                l2 = list(utilityMatrix[k3].values())
                sim_result = cosineSim(sumlist1, l1, l2)
                simMetric[k1][k2][k3] = sim_result
                pageTable[(k2, k3)] = sim_result
                pageTable[(k3, k2)] = sim_result

In [8]:
# Item - Item Collaborative Filtering
for k1, v1 in testDatasetWithoutRating.items(): # k1 user, v1 movies | test dataset user
    for k2, v2 in v1.items(): # k2 movie id | test dataset movie
        sijrjx = []
        sij = []
        for k3, v3 in simMetric[k1][k2].items():
            # threshold 0.5 to increase accuracy
            if v3 > 0.5:
                m = v3 * utilityMatrix[k3][k1]
                sijrjx.append(m) # utility movie user 
                sij.append(v3) # add similiarity
        sum_sij = sum(sij)
        sum_sijrjx = sum(sijrjx)
        # similiarity is not available. 0 
        if sum_sij == 0:
            testDatasetWithoutRating[k1][k2] = 0 # none
        # save predictions
        else:
            testDatasetWithoutRating[k1][k2] = round(sum_sijrjx / sum_sij, 2)
# list of actual rating data
list_actual = []
# list of prediction rating data
list_prediction = []
# extract actual rating data
for k1, v1 in testDataset.items():
    for k2, v2 in v1.items():
        list_actual.append(v2)
print("Original rating")
print(list_actual)
# extract prediction rating data
for k1, v1 in testDatasetWithoutRating.items():
    for k2, v2 in v1.items():
        list_prediction.append(v2)
print("Prediction rating")
print(list_prediction)
# calculate RMSE
item_item_collabo_filter_rmse = RMSE(list_actual, list_prediction)
print(f'Item - Item Collaborative Filtering, RMSE: {item_item_collabo_filter_rmse}')

Original rating
[3, 3, 5, 1, 4, 3, 3, 3, 4, 4, 4, 3, 5, 5, 4, 4, 5, 4, 5, 4, 4, 4, 4, 4, 4, 1, 4, 4, 3, 4, 3, 4, 5, 4, 4, 5, 5, 2, 3, 4, 4, 4, 3, 4, 1, 1, 3, 5, 2, 4, 4, 4, 3, 3, 3, 4, 4, 4, 3, 4, 3, 4, 1, 3, 3, 3, 1, 5, 4, 3, 3, 4, 5, 3, 1, 1, 1, 4, 4, 3, 3, 2, 3, 4, 3, 3, 3, 3, 3, 3, 1, 4, 3, 3, 3, 3, 2, 4, 3, 4, 1, 5, 3, 1, 4, 3, 4, 4, 4, 2, 3, 5, 4, 2, 3, 4, 4, 2, 4, 4, 3, 4, 4, 5, 3, 3, 3, 2, 3, 3, 3, 4, 3, 2, 3, 3, 3, 4, 3, 4, 3, 4, 4, 2, 5, 4, 2, 3, 2, 2, 2, 2, 3, 3, 1, 4, 4, 5, 5, 5, 4, 2, 3, 5, 4, 5, 5, 5, 2, 4, 5, 4, 2, 5, 4, 5, 3, 5, 2, 5, 3, 4, 4, 4, 2, 5, 4, 5, 4, 1, 3, 5, 5, 4, 3, 4, 4, 4, 3, 5, 5, 1, 4, 4, 3, 5, 4, 1, 4, 5, 3, 3, 4, 5, 3, 5, 5, 4, 5, 4, 5, 3, 3, 4, 2, 1, 4, 5, 4, 2, 3, 2, 1, 1, 1, 5, 4, 5, 3, 1, 4, 3, 3, 5, 1, 3, 4, 1, 2, 3, 5, 5, 3, 5, 4, 2, 1, 4, 2, 5, 5, 5, 4, 4, 3, 5, 3, 5, 4, 4, 5, 4, 5, 4, 4, 3, 4, 5, 5, 5, 4, 5, 4, 3, 4, 4, 1, 2, 4, 4, 4, 3, 4, 2, 4, 2, 4, 4, 2, 2, 4, 5, 4, 4, 5, 3, 2, 4, 5, 2, 4, 4, 5, 4, 4, 3, 3, 4, 4, 4, 2, 4, 1, 4, 5, 4, 3, 4,

# Bonus Tasks
# 1. Global Baseline Estimates

In [9]:
# Global Baseline Estimates

# calculate overall mean movie rating
mu_sum = 0
mu_count = 0
# trainingDataset
for k, v in trainingDataset.items():
    mu_count = mu_count + len(v)
    for (user, rating) in v:
        mu_sum = mu_sum + rating
# mu: overall mean movie rating
mu = mu_sum / mu_count

for k1, v1 in testDatasetWithoutRating_global_base.items(): # user
    # average rating of user x
    rx_star = (np.sum(list(training_user_x_global_baseline[k1].values())) / len(training_user_x_global_baseline[k1])) - mu
    for k2, v2 in v1.items(): # movie
        # average rating of item i
        ri_star = 0
        if k2 in training_item_i_global_baseline:
            ri_star = (np.sum(list(training_item_i_global_baseline[k2].values())) / len(training_item_i_global_baseline[k2])) - mu
        testDatasetWithoutRating_global_base[k1][k2] = round(mu + rx_star + ri_star, 2)

# list of prediction rating data
gbe_list_prediction = []

print("Original rating")
print(list_actual)
# extract prediction rating data
for k1, v1 in testDatasetWithoutRating_global_base.items():
    for k2, v2 in v1.items():
        gbe_list_prediction.append(v2)
print("Prediction rating")
print(gbe_list_prediction)
# calculate RMSE
global_base_est_rmse = RMSE(list_actual, gbe_list_prediction)
print(f'Global baseline estimates, RMSE: {global_base_est_rmse}')

Original rating
[3, 3, 5, 1, 4, 3, 3, 3, 4, 4, 4, 3, 5, 5, 4, 4, 5, 4, 5, 4, 4, 4, 4, 4, 4, 1, 4, 4, 3, 4, 3, 4, 5, 4, 4, 5, 5, 2, 3, 4, 4, 4, 3, 4, 1, 1, 3, 5, 2, 4, 4, 4, 3, 3, 3, 4, 4, 4, 3, 4, 3, 4, 1, 3, 3, 3, 1, 5, 4, 3, 3, 4, 5, 3, 1, 1, 1, 4, 4, 3, 3, 2, 3, 4, 3, 3, 3, 3, 3, 3, 1, 4, 3, 3, 3, 3, 2, 4, 3, 4, 1, 5, 3, 1, 4, 3, 4, 4, 4, 2, 3, 5, 4, 2, 3, 4, 4, 2, 4, 4, 3, 4, 4, 5, 3, 3, 3, 2, 3, 3, 3, 4, 3, 2, 3, 3, 3, 4, 3, 4, 3, 4, 4, 2, 5, 4, 2, 3, 2, 2, 2, 2, 3, 3, 1, 4, 4, 5, 5, 5, 4, 2, 3, 5, 4, 5, 5, 5, 2, 4, 5, 4, 2, 5, 4, 5, 3, 5, 2, 5, 3, 4, 4, 4, 2, 5, 4, 5, 4, 1, 3, 5, 5, 4, 3, 4, 4, 4, 3, 5, 5, 1, 4, 4, 3, 5, 4, 1, 4, 5, 3, 3, 4, 5, 3, 5, 5, 4, 5, 4, 5, 3, 3, 4, 2, 1, 4, 5, 4, 2, 3, 2, 1, 1, 1, 5, 4, 5, 3, 1, 4, 3, 3, 5, 1, 3, 4, 1, 2, 3, 5, 5, 3, 5, 4, 2, 1, 4, 2, 5, 5, 5, 4, 4, 3, 5, 3, 5, 4, 4, 5, 4, 5, 4, 4, 3, 4, 5, 5, 5, 4, 5, 4, 3, 4, 4, 1, 2, 4, 4, 4, 3, 4, 2, 4, 2, 4, 4, 2, 2, 4, 5, 4, 4, 5, 3, 2, 4, 5, 2, 4, 4, 5, 4, 4, 3, 3, 4, 4, 4, 2, 4, 1, 4, 5, 4, 3, 4,

# 2. Item-Item Adjusted Cosine Sim

In [10]:
# adjusted cosine sim 
def adjCosineSim(ry_star, list1, list2):
    # average rating of
    # the average rating of user y
    r_star_1 = []
    for l1 in list1:
        if l1 != 0:
            r_star_1.append(l1 - ry_star)
        else:
            r_star_1.append(l1)
    # the average rating of user y
    r_star_2 = []
    for l2 in list2:
        if l2 != 0:
            r_star_2.append(l2 - ry_star)
        else:
            r_star_2.append(l2)
    # the dot product of rates
    sumList = np.dot(r_star_1, r_star_2)
    if sumList == 0:
        return 0
    sumlist1 = np.sum(np.square(r_star_1))
    if sumlist1 == 0:
        return 0
    sqr_list1 = sqrt(sumlist1)
    sumlist2 = np.sum(np.square(r_star_2))
    if sumlist2 == 0:
        return 0
    sqr_list2 = sqrt(sumlist2)
    # adjusted cosine similarity
    sim = sumList / (sqr_list1 * sqr_list2)
    return sim

In [11]:
# similarity matric
ajdSimMetric = dict.fromkeys(sorted(unique_user_x_list), dict()) # user 100

for k1, v1 in user_x_movie_dict.items(): # user ID / movie ID
    ajdSimMetric[k1] = dict.fromkeys(sorted(v1), dict()) # movieId of test dataset
    for v in v1:
        temp_dict = dict.fromkeys(sorted(training_user_x_movie_list[k1]), 0) #  # movieId of training dataset rated by same user
        if not v in ajdSimMetric:
            ajdSimMetric[k1][v] = temp_dict
        else:
            ajdSimMetric[k1][v].update(temp_dict)
# pagenation
adj_pageTable = dict()

for k1, v1 in ajdSimMetric.items(): # user
    ry_star = (np.sum(list(training_user_x_global_baseline[k1].values())) / len(training_user_x_global_baseline[k1]))
    for k2, v2 in v1.items(): # first movie ID, testdata set
        l1 = list(utilityMatrix[k2].values())
        for k3, v3 in v2.items(): # second movie ID, training set 
            # if movie1 and movie2 are same, similiarity will be 1. No need to calculate
            if k2 == k3:
                ajdSimMetric[k1][k2][k3] = 1.0
                adj_pageTable[(k2, k3)] = 1.0
                adj_pageTable[(k3, k2)] = 1.0
            # page hit, no need to calculate
            elif (k2, k3) in adj_pageTable and adj_pageTable[(k2, k3)] != 0.0:
                ajdSimMetric[k1][k2][k3] = adj_pageTable[(k2, k3)]
            # page hit with inverted indexes, no need to calculate
            elif (k3, k2) in adj_pageTable and adj_pageTable[(k3, k2)] != 0.0:
                ajdSimMetric[k1][k2][k3] = adj_pageTable[(k3, k2)]
            # no hit. Calculation needed
            else:
                l2 = list(utilityMatrix[k3].values())
                sim_result = adjCosineSim(ry_star, l1, l2)
                ajdSimMetric[k1][k2][k3] = sim_result
                adj_pageTable[(k2, k3)] = sim_result
                adj_pageTable[(k3, k2)] = sim_result

In [12]:
for k1, v1 in testDatasetWithoutRating_adj.items(): # k1 user, v1 movies | test dataset user
    for k2, v2 in v1.items(): # k2 movie id | test dataset movie
        sijrjx = []
        sij = []
        for k3, v3 in ajdSimMetric[k1][k2].items():      
            if v3 > 0.2:
                m = v3 * utilityMatrix[k3][k1]
                sijrjx.append(m) # utility movie user 
                sij.append(v3)
        sum_sij = sum(sij)
        sum_sijrjx = sum(sijrjx)
        # similiarity is not available. 0 
        if sum_sij == 0:
            testDatasetWithoutRating_adj[k1][k2] = 0 # none
        else:
            testDatasetWithoutRating_adj[k1][k2] = round(sum_sijrjx / sum_sij, 2)
# list of prediction rating data
adj_list_prediction = []

print("Original rating")
print(list_actual)
# extract prediction rating data
for k1, v1 in testDatasetWithoutRating_adj.items():
    for k2, v2 in v1.items():
        adj_list_prediction.append(v2)
print("Prediction rating")
print(adj_list_prediction)
# calculate RMSE
adj_cos_rmse = RMSE(list_actual, adj_list_prediction)
print(f'Item - Item Collaborative filtering with adjusted cos sim, RMSE: {adj_cos_rmse}')

Original rating
[3, 3, 5, 1, 4, 3, 3, 3, 4, 4, 4, 3, 5, 5, 4, 4, 5, 4, 5, 4, 4, 4, 4, 4, 4, 1, 4, 4, 3, 4, 3, 4, 5, 4, 4, 5, 5, 2, 3, 4, 4, 4, 3, 4, 1, 1, 3, 5, 2, 4, 4, 4, 3, 3, 3, 4, 4, 4, 3, 4, 3, 4, 1, 3, 3, 3, 1, 5, 4, 3, 3, 4, 5, 3, 1, 1, 1, 4, 4, 3, 3, 2, 3, 4, 3, 3, 3, 3, 3, 3, 1, 4, 3, 3, 3, 3, 2, 4, 3, 4, 1, 5, 3, 1, 4, 3, 4, 4, 4, 2, 3, 5, 4, 2, 3, 4, 4, 2, 4, 4, 3, 4, 4, 5, 3, 3, 3, 2, 3, 3, 3, 4, 3, 2, 3, 3, 3, 4, 3, 4, 3, 4, 4, 2, 5, 4, 2, 3, 2, 2, 2, 2, 3, 3, 1, 4, 4, 5, 5, 5, 4, 2, 3, 5, 4, 5, 5, 5, 2, 4, 5, 4, 2, 5, 4, 5, 3, 5, 2, 5, 3, 4, 4, 4, 2, 5, 4, 5, 4, 1, 3, 5, 5, 4, 3, 4, 4, 4, 3, 5, 5, 1, 4, 4, 3, 5, 4, 1, 4, 5, 3, 3, 4, 5, 3, 5, 5, 4, 5, 4, 5, 3, 3, 4, 2, 1, 4, 5, 4, 2, 3, 2, 1, 1, 1, 5, 4, 5, 3, 1, 4, 3, 3, 5, 1, 3, 4, 1, 2, 3, 5, 5, 3, 5, 4, 2, 1, 4, 2, 5, 5, 5, 4, 4, 3, 5, 3, 5, 4, 4, 5, 4, 5, 4, 4, 3, 4, 5, 5, 5, 4, 5, 4, 3, 4, 4, 1, 2, 4, 4, 4, 3, 4, 2, 4, 2, 4, 4, 2, 2, 4, 5, 4, 4, 5, 3, 2, 4, 5, 2, 4, 4, 5, 4, 4, 3, 3, 4, 4, 4, 2, 4, 1, 4, 5, 4, 3, 4,

In [13]:
print(f'Item - Item Collaborative Filtering with cos sim, RMSE: {item_item_collabo_filter_rmse}')
print(f'Item - Item Collaborative Filtering with adjusted cos sim, RMSE: {adj_cos_rmse}')
print(f'Global baseline estimates, RMSE: {global_base_est_rmse}')


Item - Item Collaborative Filtering with cos sim, RMSE: 2.6393
Item - Item Collaborative Filtering with adjusted cos sim, RMSE: 1.1467
Global baseline estimates, RMSE: 0.946


# Compare all results and give reasoning

Item - Item Collaborative Filtering with cos sim, RMSE: 2.6393

Item - Item Collaborative Filtering with adjusted cos sim, RMSE: 1.1467

Global baseline estimates, RMSE: 0.946

Global Baseline estimation returned most accurate prediction according to RMSE value. And Item - Item Collaborative Filtering with adjusted cosine similarity and with cosine similarity respectively. Global baseline estimation gives the lowest RMSE because it is based on adjusting the average value with each factor later. However, each individual prediction value is bit far away from actual value. Item - Item Collaborative Filtering with adjusted cosine similarity gives 1.14 RMSE when it has low threshold value. It is due to the nature of the equation takes off the average rating of user from each rating. This gives higher accuracy but return lower similarity values. Lastly, Item - Item Collaborative Filtering with cosine similarity has highest RMSE because it does not consider different rating schemes of different user. Also, predicting ratings that does not have any ratings from other users gives 0 similarity which decrease the accuracy sevearly.

